# TA - Entregable 1/Entregable 2

Para nuestro proyecto, primero que nada necesitamos extraer la data historica de las principales indeces de los 5 países con más casos de coronavirus (al 06 de mayo del 2020):
   
    *Estados Unidos (S&P 500) 
    *Italia (FTSE-MIB)
    *España (IBEX 35)
    *Reino Unido (FTSE 100)
    *Alemania (DAX)
    
Además, analizaremos el principal índice de Perú para comparar el comportamiento económico de estos países con la situación nacional:

    *Perú (S&P/BVL)
 
Esto lo haremos usando la información en la plataforma https://www.investing.com/instruments/HistoricalDataAjax. Sin embargo, en el caso de Perú la data histórica solo está disponible desde el año 2011, así que para este usaremos https://finance.yahoo.com/.

## Datasets financieras

Nuestra data respecto a las bolsas de valores se dividirá en dos datasets principales:

    *tendencias_19_20: valores de cierre diarios de cada país desde el 06 de mayo de 2019 hasta el 06 de mayo de 2020

    *tendencias_dec_jul: valores de cierre diarios de cada país en el periodo del 01 de diciembre al 31 de julio desde el  
    2006 hasta el año actual(en este último solo llegaaran hasta el día anterior al presente, por razones obvias)
    
Nota: los valores de los países están en diferentes unidades (monedas), pero dado que la comparación que buscamos hacer es de cada país sus propias tendencias en distintos periodos de tiempo, esto no es un problema.

### tendencias_19_20

Descripción de tendencias_19_20:

    *date: fecha de las operaciones, tipo datetime
    *estados_unidos: valor de cierre del S&P 500 en la fecha indicada (en USD), tipo float
    *italia: valor de cierre del FTSE-MIB en la fecha indicada (en EUR), tipo float
    *españa: valor de cierre del IBEX 35 en la fecha indicada (en EUR), tipo float
    *reino_unido: valor de cierre del FTSE 100 en la fecha indicada (en GBP), tipo float
    *alemania: valor de cierre del DAX en la fecha indicada (en EUR), tipo float
    *peru: valor de cierre del S&P/BVL en la fecha indicada (en PEN), tipo float    

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from pymongo import MongoClient

Primero extraeremos la información de los índices disponibes en investing.com

In [2]:
header = {
  'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36' ,
  'X-Requested-With': 'XMLHttpRequest'
}

llaves = {'ESTADOS_UNIDOS':['166','2030167','S&amp;P 500 Historical Data'],
          'ITALIA':['177','2030177','FTSE MIB HistorialData'],
          'ESPAÑA':['174','2030174','IBEX 35 Historical Data'],
          'REINO_UNIDO':['27','2030166','FTSE 100 Historical Data'],
          'ALEMANIA':['172','2030173','DAX Historical Data']}


In [3]:
url = "https://www.investing.com/instruments/HistoricalDataAjax"
sopas = {}

for key, item in llaves.items():
    form = {'curr_id': item[0], 'smlID': item[1],'header': item[2], 'st_date': '05/07/2019',
        'end_date': '05/06/2020', 'interval_sec': 'Daily', 'sort_col': 'date', 'sort_ord': 'DESC','action': 'historical_data'}
    response = requests.post(url,data=form,headers=header).text.replace('\n', '')
    soup = BeautifulSoup(response,'html.parser')
    sopas[key] = soup
    
len(sopas)

5

In [4]:
info = {}

for key, soup in sopas.items():
    holder={'date':[], 'close': []}

    table =soup.find("table", id="curr_table")
    dataraw = table.tbody.find_all("tr")
    for data in dataraw:
        cosas = data.find_all('td')
        holder['date'].append(cosas[0].text)
        holder['close'].append(cosas[1].text)
        
    info[key]=holder

In [5]:
dfs_info={}

for key, dicti in info.items():
    df = pd.DataFrame(dicti)
    dfs_info[key]=df

In [6]:
len(dfs_info)

5

Veamos uno de los dataframes

In [7]:
dfs_info['ITALIA']

,date,close
0,"May 06, 2020","17,159.31"
1,"May 05, 2020","17,387.38"
2,"May 04, 2020","17,035.61"
3,"Apr 30, 2020","17,690.49"
4,"Apr 29, 2020","18,067.29"
...,...,...
248,"May 13, 2019","20,593.72"
249,"May 10, 2019","20,874.78"
250,"May 09, 2019","20,817.16"
251,"May 08, 2019","21,203.86"


Por sentido prácticos, es necesario establecer la columna 'date' como índice y de tipo datetime. Y a la columna 'close', ponerle el nombre del país respectivo y establecerlo como tipo float

In [8]:
dfs_list=[]
for key, data in dfs_info.items():
    
    data['date'] = pd.to_datetime(data['date']) 
    data1 = data.set_index('date', drop=True)
    data1.columns = [key.lower()]
    
    data1[key.lower()] = [float(x.replace(',','')) for x in data1[key.lower()]]
    
    dfs_list.append(data1)
    
data1.head()

,alemania
date,
2020-05-06,10606.20
2020-05-05,10729.46
2020-05-04,10466.80
2020-04-30,10861.64
2020-04-29,11107.74


Ahora, extreeremos la información del índice de Perú

In [9]:
from datetime import datetime, timedelta

In [10]:
ticket_p = {'PERU':'^SPBLPGPT'}

final_p = datetime(2020,5,7)
hasta_p = int(final_p.timestamp())

inicio_p = final_p - timedelta(days=366)
desde_p = int(inicio_p.timestamp())

intervalo_p='1d'

url_base='https://query1.finance.yahoo.com/v7/finance/download/'

In [11]:
url_p = url_base + ticket_p['PERU'] +'?period1=' + str(desde_p) + '&period2=' + str(hasta_p) + '&interval=' + intervalo_p + '&events=history'
        
df_p = pd.read_csv(url_p, parse_dates=['Date'])

df_p.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-05-07,20765.949219,20776.779297,20594.759766,20688.019531,20688.019531,0.0
1,2019-05-08,20694.269531,20701.529297,20430.449219,20438.560547,20438.560547,0.0
2,2019-05-09,20510.619141,20530.669922,20210.009766,20213.300781,20213.300781,0.0
3,2019-05-10,20216.349609,20216.349609,20046.519531,20093.300781,20093.300781,0.0
4,2019-05-13,20157.699219,20186.919922,19943.019531,19951.960938,19951.960938,0.0


In [12]:
df_peru = df_p.copy()
df_peru = df_peru.iloc[:,[0,4]]

df_peru.columns = ['date','peru']
df_peru['peru'] = [round(x,2) for x in df_peru['peru']]
df_peru1 = df_peru.set_index('date')

dfs_list.append(df_peru1)

df_peru1

,peru
date,
2019-05-07,20688.02
2019-05-08,20438.56
2019-05-09,20213.30
2019-05-10,20093.30
2019-05-13,19951.96
...,...
2020-04-30,14779.28
2020-05-01,NaN
2020-05-04,14722.30


Ahora que el df de Perú tiene el mismo formato que los otros, podemos concatenarlos

In [13]:
tendencias = pd.concat(dfs_list,axis=1)

tendencias

,estados_unidos,italia,españa,reino_unido,alemania,peru
date,,,,,,
2019-05-07,2884.05,21219.14,9235.1,7260.47,12092.74,20688.02
2019-05-08,2879.42,21203.86,9227.0,7271.00,12179.93,20438.56
2019-05-09,2870.72,20817.16,9095.2,7207.41,11973.92,20213.30
2019-05-10,2881.40,20874.78,9117.5,7203.29,12059.83,20093.30
2019-05-13,2811.87,20593.72,9046.8,7163.68,11876.65,19951.96
...,...,...,...,...,...,...
2020-04-30,2912.43,17690.49,6922.3,5901.21,10861.64,14779.28
2020-05-01,2830.71,NaN,NaN,5763.06,NaN,NaN
2020-05-04,2842.74,17035.61,6673.3,5753.78,10466.80,14722.30


In [14]:
tendencias.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 259 entries, 2019-05-07 to 2020-05-06
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   estados_unidos  253 non-null    float64
 1   italia          253 non-null    float64
 2   españa          256 non-null    float64
 3   reino_unido     255 non-null    float64
 4   alemania        252 non-null    float64
 5   peru            247 non-null    float64
dtypes: float64(6)
memory usage: 14.2 KB


Vemos que todas nuestras columnas tienen más de un valor nulo, esto se debe a que ya sea por un feriado o algún evento nacional, esa bolsa de valores no opero ese día.

De hecho, ninguna bolsa de valores trabaja durante los fines de semana, por eso es que se pueden observar saltos consistentes de 2 días cada cierto intervalo de registrods. Sin embargo, por razones de continuidad al momento de hacer el análsis necesario, necesitamos agregar estos registros, y luego atribuirles valores.

Primero, nos encargaremos de agregar las fechas faltantes, y dado que estos son días en los que la bolsa no operó, el valor permanece igual al de la última fecha activa

In [15]:
tendencias1 = tendencias.resample('D').ffill()

tendencias1.tail(7)

,estados_unidos,italia,españa,reino_unido,alemania,peru
date,,,,,,
2020-04-30,2912.43,17690.49,6922.3,5901.21,10861.64,14779.28
2020-05-01,2830.71,NaN,NaN,5763.06,NaN,NaN
2020-05-02,2830.71,NaN,NaN,5763.06,NaN,NaN
2020-05-03,2830.71,NaN,NaN,5763.06,NaN,NaN
2020-05-04,2842.74,17035.61,6673.3,5753.78,10466.80,14722.30
2020-05-05,2868.44,17387.38,6747.7,5849.42,10729.46,14900.48
2020-05-06,2848.42,17159.31,6671.7,5853.76,10606.20,14953.76


In [16]:
tendencias1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 366 entries, 2019-05-07 to 2020-05-06
Freq: D
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   estados_unidos  360 non-null    float64
 1   italia          356 non-null    float64
 2   españa          361 non-null    float64
 3   reino_unido     362 non-null    float64
 4   alemania        355 non-null    float64
 5   peru            345 non-null    float64
dtypes: float64(6)
memory usage: 20.0 KB


Vemos que ahora si tenemos la cantidad de entradas correspondientes a un año (bisiesto), pero todavía hay valores vacíos, correspondientes a los feriados ocurridos durante días de semana

Estos los llenaremos usando el miso método, ya que la misma lógica se aplica

In [17]:
tendencias_19_20= tendencias1.fillna(method='ffill')
    
tendencias_19_20.tail(7)

,estados_unidos,italia,españa,reino_unido,alemania,peru
date,,,,,,
2020-04-30,2912.43,17690.49,6922.3,5901.21,10861.64,14779.28
2020-05-01,2830.71,17690.49,6922.3,5763.06,10861.64,14779.28
2020-05-02,2830.71,17690.49,6922.3,5763.06,10861.64,14779.28
2020-05-03,2830.71,17690.49,6922.3,5763.06,10861.64,14779.28
2020-05-04,2842.74,17035.61,6673.3,5753.78,10466.80,14722.30
2020-05-05,2868.44,17387.38,6747.7,5849.42,10729.46,14900.48
2020-05-06,2848.42,17159.31,6671.7,5853.76,10606.20,14953.76


In [18]:
tendencias_19_20.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 366 entries, 2019-05-07 to 2020-05-06
Freq: D
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   estados_unidos  366 non-null    float64
 1   italia          366 non-null    float64
 2   españa          366 non-null    float64
 3   reino_unido     366 non-null    float64
 4   alemania        366 non-null    float64
 5   peru            366 non-null    float64
dtypes: float64(6)
memory usage: 20.0 KB


Listo, ahora ya tenemos todos los registros sin valores vacíos

Primero lo guardaremos en un Excel

In [19]:
tendencias_19_20.to_excel('tendencias_19_20.xlsx')

Pero además lo almacenaremos en una colección de MongoDB

In [20]:
tendencias_19_20.index = [str(x) for x in tendencias_19_20.index]
tendencias_19_20_ =tendencias_19_20.reset_index().rename(columns={'index':'date'})
#Es necesario pasar las fechas a strings porque mongo no trabaja bien con datatimes
tendencias_19_20_.dtypes

date               object
estados_unidos    float64
italia            float64
españa            float64
reino_unido       float64
alemania          float64
peru              float64
dtype: object

In [21]:
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient["TA"]
mycol = mydb["tend_19_20"]

mycol.drop()

In [22]:
records = json.loads(tendencias_19_20_.T.to_json()).values()
mycol.insert_many(records)

### tendencias_dec_jul

Descripción de tendencias_dec_jul:

    *year: el año al que hace referencia la sección, tipo int 
    *date: fecha de las operaciones, tipo datetime
    *estados_unidos: valor de cierre del S&P 500 en la fecha indicada (en USD), tipo float
    *italia: valor de cierre del FTSE-MIB en la fecha indicada (en EUR), tipo float
    *españa: valor de cierre del IBEX 35 en la fecha indicada (en EUR), tipo float
    *reino_unido: valor de cierre del FTSE 100 en la fecha indicada (en GBP), tipo float
    *alemania: valor de cierre del DAX en la fecha indicada (en EUR), tipo float
    *peru: valor de cierre del S&P/BVL en la fecha indicada (en PEN), tipo float    

Solo es necesario correr las celdas comentadas si es la primera vez que usas el notebook en tu servidor

In [23]:
'''
t_inicio = '12/01/'
t_final ='07/31/'

#Vamos ha hacer los años hasta el 2019 separados a este, porque el de este año vaos a tener que estar acualizandolo,
#Y así no tenemosque correr tooodos los for todo el tiempo
conjunto = {}
for i in range(2006,2019):   

    anual={}
    for key, item in llaves.items():
        holder={'date':[], 'close': []}
        
        inicio = t_inicio + str(i)
        final = t_final + str(i+1)
        
        form = {'curr_id': item[0], 'smlID': item[1],'header': item[2], 'st_date': inicio,
                'end_date': final, 'interval_sec': 'Daily', 'sort_col': 'date', 'sort_ord': 'DESC','action': 'historical_data'}
        response = requests.post(url,data=form,headers=header).text.replace('\n', '')
        soup = BeautifulSoup(response,'html.parser')
        
        table =soup.find("table", id="curr_table")
        dataraw = table.tbody.find_all("tr")
        for data in dataraw:
            cosas = data.find_all('td')
            holder['date'].append(cosas[0].text)
            holder['close'].append(cosas[1].text)
        
        anual[key]=holder
    conjunto[str(i+1)]=anual
'''

'\nt_inicio = \'12/01/\'\nt_final =\'07/31/\'\n\n#Vamos ha hacer los años hasta el 2019 separados a este, porque el de este año vaos a tener que estar acualizandolo,\n#Y así no tenemosque correr tooodos los for todo el tiempo\nconjunto = {}\nfor i in range(2006,2019):   \n\n    anual={}\n    for key, item in llaves.items():\n        holder={\'date\':[], \'close\': []}\n        \n        inicio = t_inicio + str(i)\n        final = t_final + str(i+1)\n        \n        form = {\'curr_id\': item[0], \'smlID\': item[1],\'header\': item[2], \'st_date\': inicio,\n                \'end_date\': final, \'interval_sec\': \'Daily\', \'sort_col\': \'date\', \'sort_ord\': \'DESC\',\'action\': \'historical_data\'}\n        response = requests.post(url,data=form,headers=header).text.replace(\'\n\', \'\')\n        soup = BeautifulSoup(response,\'html.parser\')\n        \n        table =soup.find("table", id="curr_table")\n        dataraw = table.tbody.find_all("tr")\n        for data in dataraw:\n    

In [24]:
'''
#Para no tener que estar corriendolo a cada rato vamos a guardalro en un json
with open('2007_2019.json', 'w') as fp:
    json.dump(conjunto, fp)
'''

"\n#Para no tener que estar corriendolo a cada rato vamos a guardalro en un json\nwith open('2007_2019.json', 'w') as fp:\n    json.dump(conjunto, fp)\n"

In [25]:
#Ahora recuperamos nuestra nuestra info
with open('2007_2019.json', 'r') as fp:
    conjunto = json.load(fp)
    
len(conjunto['2007']['ITALIA']['close'])

172

In [26]:
#Ahora saquemos la info de este año

final = datetime.today() - timedelta(days=1)
final = final.strftime('%m/%d/%Y')

conj2020={}
for key, item in llaves.items():
    
    venn2020={'date':[],'close':[]}
    
    form = {'curr_id': item[0], 'smlID': item[1],'header': item[2], 'st_date': '12/01/2019',
                'end_date': final, 'interval_sec': 'Daily', 'sort_col': 'date', 'sort_ord': 'DESC','action': 'historical_data'}
    response = requests.post(url,data=form,headers=header).text.replace('\n', '')
    soup = BeautifulSoup(response,'html.parser')
        
    table =soup.find("table", id="curr_table")
    dataraw = table.tbody.find_all("tr")
    
    for data in dataraw:
        cosas = data.find_all('td')
        venn2020['date'].append(cosas[0].text)
        venn2020['close'].append(cosas[1].text)
        
    conj2020[key]=venn2020
    
conjunto['2020'] = conj2020

In [27]:
grandes=[]
for keys, itemo in conjunto.items():
    pedazos=[]
    for llavecita, cosa in itemo.items():
        data = pd.DataFrame(cosa)
        data['date'] = pd.to_datetime(data['date']) 
        data1 = data.set_index('date', drop=True)
        data1.columns = [llavecita.lower()]
        pedazos.append(data1)
    
    #Aplicamos los mismos métodos de resampling y llenado de datos que usamos en tendencias_19_20
    junto= pd.concat(pedazos,axis=1)
    junto= junto.resample('D').ffill()
    junto =junto.fillna(method='ffill')
    
    #Para los datos vaciós que queden (aquellos en los cuales el 01 de dicimebre fue fin de semana) usaremos 'bfill'
    junto =junto.fillna(method='bfill')    
    
    año = [int(keys) for x in junto.index ]
    juntos = junto.reset_index()
    juntos.insert(0,'year',año)
    juntos1 = juntos.set_index(['year','date'])
    for col in juntos1.columns:
        juntos1[col]=[float(x.replace(',','')) for x in juntos1[col]]
    
    grandes.append(juntos1)

len(grandes)

14

In [28]:
tendencias_sinpe = pd.concat(grandes)
                            
tendencias_sinpe

estados_unidos    italia   españa  reino_unido  alemania
year date                                                                
2007 2006-12-01         1396.71  39949.00  13660.6      6021.50   6241.13
     2006-12-02         1396.71  39949.00  13660.6      6021.50   6241.13
     2006-12-03         1396.71  39949.00  13660.6      6021.50   6241.13
     2006-12-04         1409.12  40155.00  13846.4      6050.40   6295.23
     2006-12-05         1414.76  40408.00  14032.5      6086.40   6372.80
...                         ...       ...      ...          ...       ...
2020 2020-07-10         3185.04  19767.60   7321.1      6095.41  12633.71
     2020-07-11         3185.04  19767.60   7321.1      6095.41  12633.71
     2020-07-12         3185.04  19767.60   7321.1      6095.41  12633.71
     2020-07-13         3155.22  20003.25   7426.9      6176.19  12799.97
     2020-07-14         3197.52  19879.75   7352.0      6179.75  12697.36

[3377 rows x 5 columns]

In [29]:
tendencias_sinpe.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3377 entries, (2007, Timestamp('2006-12-01 00:00:00')) to (2020, Timestamp('2020-07-14 00:00:00'))
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   estados_unidos  3377 non-null   float64
 1   italia          3377 non-null   float64
 2   españa          3377 non-null   float64
 3   reino_unido     3377 non-null   float64
 4   alemania        3377 non-null   float64
dtypes: float64(5)
memory usage: 168.4 KB


Ahora sacaremos la info de perú

In [30]:
#Como este no demora muco en correr, vamos a incluir el 2020 aquí
df_principal =''
for i in range(2006,2020):   
    holder={'date':[], 'close': []}
    if i == 2019:
        final_pe = datetime.today() - timedelta(days=1)
    else:
        final_pe = datetime(i+1,8,1)
    
    inicio_pe = datetime(i,12,1)
        
    hasta_pe = int(final_pe.timestamp())
    desde_pe = int(inicio_pe.timestamp())
    
    url_pe = url_base + ticket_p['PERU'] +'?period1=' + str(desde_pe) + '&period2=' + str(hasta_pe) + '&interval=' + intervalo_p + '&events=history'
        
    df_pe = pd.read_csv(url_pe, parse_dates=['Date'])

    df_perux = df_pe.copy()
    df_perux = df_perux.iloc[:,[0,4]]

    df_perux.columns = ['date','peru']
    df_perux['peru'] = [round(x,2) for x in df_perux['peru']]
    df_peru1x = df_perux.set_index('date')
    
    df_peru1x = df_peru1x.resample('D').ffill()
    df_peru1x =df_peru1x.fillna(method='ffill')
    
    #Para los datos vaciós que queden (aquellos en los cuales el 01 de dicimebre fue fin de semana) usaremos 'bfill'
    df_peru1x =df_peru1x.fillna(method='bfill')   
    
    año = [i+1 for x in df_peru1x.index]
    df_peru2x = df_peru1x.reset_index()
    df_peru2x.insert(0,'year',año)
    df_peru3x = df_peru2x.set_index(['year','date'])
    
    if i==2006:
        df_principal = df_peru3x.copy()
    else:
        df_principal = pd.concat([df_principal,df_peru3x])


In [31]:
tendencias_dec_jul = pd.concat([tendencias_sinpe,df_principal],axis=1)
tendencias_dec_jul

estados_unidos    italia   españa  reino_unido  alemania  \
year date                                                                   
2007 2006-12-01         1396.71  39949.00  13660.6      6021.50   6241.13   
     2006-12-02         1396.71  39949.00  13660.6      6021.50   6241.13   
     2006-12-03         1396.71  39949.00  13660.6      6021.50   6241.13   
     2006-12-04         1409.12  40155.00  13846.4      6050.40   6295.23   
     2006-12-05         1414.76  40408.00  14032.5      6086.40   6372.80   
...                         ...       ...      ...          ...       ...   
2020 2020-07-10         3185.04  19767.60   7321.1      6095.41  12633.71   
     2020-07-11         3185.04  19767.60   7321.1      6095.41  12633.71   
     2020-07-12         3185.04  19767.60   7321.1      6095.41  12633.71   
     2020-07-13         3155.22  20003.25   7426.9      6176.19  12799.97   
     2020-07-14         3197.52  19879.75   7352.0      6179.75  12697.36   

                     peru  
year date                  
2007 2006-12-01  11535.41  
     2006-12-02  11535.41  
     2006-12-03  11535.41  
     2006-12-04  11649.51  
     2006-12-05  11713.62  
...                   ...  
2020 2020-07-10  16671.15  
     2020-07-11  16671.15  
     2020-07-12  16671.15  
     2020-07-13  16646.36  
     2020-07-14  16574.72  

[3377 rows x 6 columns]

In [32]:
tendencias_dec_jul.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3377 entries, (2007, Timestamp('2006-12-01 00:00:00')) to (2020, Timestamp('2020-07-14 00:00:00'))
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   estados_unidos  3377 non-null   float64
 1   italia          3377 non-null   float64
 2   españa          3377 non-null   float64
 3   reino_unido     3377 non-null   float64
 4   alemania        3377 non-null   float64
 5   peru            3376 non-null   float64
dtypes: float64(6)
memory usage: 194.8 KB


Parece que hay un valor nulo en 'peru'

In [33]:
tendencias_dec_jul[tendencias_dec_jul.peru.isnull()]

,,estados_unidos,italia,españa,reino_unido,alemania,peru
year,date,,,,,,
2014,2013-12-01,1800.9,19091.3,9745.5,6595.33,9401.96,NaN


El 1 de dicimebre de 2013 fue domingo, curiosamente todos mercados aparte de Perú parecen haber operado ese día. UYa que sólo es un dato, usaremos el método 'bfill' para llenarlo

In [34]:
tendencias_dec_jul = tendencias_dec_jul.fillna(method='bfill')

tendencias_dec_jul.sort_index(level=['year','date'], ascending=True, inplace=True)
tendencias_dec_jul.loc[(2014,'2013-12-01'):(2014,'2013-12-03'), :]

estados_unidos    italia  españa  reino_unido  alemania  \
year date                                                                  
2014 2013-12-01         1800.90  19091.30  9745.5      6595.33   9401.96   
     2013-12-02         1800.90  18732.56  9745.5      6595.33   9401.96   
     2013-12-03         1795.15  18366.75  9605.0      6532.43   9223.40   

                     peru  
year date                  
2014 2013-12-01  15142.06  
     2013-12-02  15142.06  
     2013-12-03  14767.72

In [35]:
tendencias_dec_jul.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3377 entries, (2007, Timestamp('2006-12-01 00:00:00')) to (2020, Timestamp('2020-07-14 00:00:00'))
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   estados_unidos  3377 non-null   float64
 1   italia          3377 non-null   float64
 2   españa          3377 non-null   float64
 3   reino_unido     3377 non-null   float64
 4   alemania        3377 non-null   float64
 5   peru            3377 non-null   float64
dtypes: float64(6)
memory usage: 355.4 KB


Listo

In [36]:
tendencias_dec_jul.to_excel('tendencias_dec_jul.xlsx')

Rstos también los almacenaremos en un MongoDB

In [37]:
tendencias_dec_jul_=tendencias_dec_jul.reset_index()
tendencias_dec_jul_['date']=[str(x) for x in tendencias_dec_jul_['date']]
#Es necesario pasar las fechas a strings porque mongo no trabaja bien con datatimes
tendencias_dec_jul_.dtypes

year                int64
date               object
estados_unidos    float64
italia            float64
españa            float64
reino_unido       float64
alemania          float64
peru              float64
dtype: object

In [38]:
mycol = mydb["tend_dec_jul"]

mycol.drop()

records = json.loads(tendencias_dec_jul_.T.to_json()).values()
mycol.insert_many(records)